In [1]:
# Data processing
import iris
import iris.analysis
import iris.coord_categorisation
import warnings
warnings.filterwarnings('ignore', module='iris')
import numpy as np
from pathlib import Path
# Visualization
import cartopy.util
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import FuncFormatter
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)
    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
plt.rcParams['mathtext.default'] = 'regular'

In [2]:
# CONSTANTS
EXPS = {'BASE OLD':'xojnd',
        'BASE'    :'xojng',
        'CHEM'    :'xojnh',
        'MARI'    :'xojni',
        'FIRE'    :'xojnc',
        'FULL'    :'xojnl'}

In [3]:
def mmr_to_ppb(molar_mass, air_mass=28.97):
    """Convert gas molar mass to mole fraction in ppb."""
    return air_mass / molar_mass * 1e9

In [4]:
VARS = {'no': {'tex': 'NO', 'molar_mass': 30.006, "noy": True},
 'no2': {'tex': '$NO_2$', 'molar_mass': 46.006, "noy": True},
 'hono': {'tex': 'HONO', 'molar_mass': 47.013, "noy": True},
 'ho2no2': {'tex': '$HO_2NO_2$', 'molar_mass': 79.01224, "noy": True},
 'hno3': {'tex': '$HNO_3$', 'molar_mass': 63.012, "noy": True},
 'n2o5': {'tex': 'N_2O_5', 'molar_mass': 108.01, "noy": True},
 'pan': {'tex': 'PAN', 'molar_mass': 121.0489, "noy": True},
 'ppan': {'tex': 'PPN', 'molar_mass': 135.0755, "noy": True},
 'meono2': {'tex': '$MeONO_2$', 'molar_mass': 77.0394, "noy": True},
 'etono2': {'tex': '$EtONO_2$', 'molar_mass': 91.066, "noy": True},
 'nprono2': {'tex': '$nPrONO_2$', 'molar_mass': 105.0926, "noy": True},
 'iprono2': {'tex': '$iPrONO_2$', 'molar_mass': 105.0926, "noy": True},
 'nox': {'tex': '$NO_x$',"noy": False},
 'noy': {'tex': '$NO_y$',"noy": False},
'ch4':{"tex": '$CH_4$', "molar_mass": 16.0425,"noy": False},
       'o3':{"tex": '$O_3$', "molar_mass": 47.997, "noy": False},

       }

for k, v in VARS.items():
    try:
        VARS[k]["mmr_to_ppb"] = mmr_to_ppb(VARS[k]["molar_mass"])
    except KeyError:
        pass

In [5]:
# Choose experiments
base_exp_name = 'BASE'
sens_exp_name = 'MARI'

In [6]:
# Read data
path_to_ukca = Path.cwd().parent / 'raw'
base_cl = iris.cube.CubeList()
sens_cl = iris.cube.CubeList()
for ivar, vardict in VARS.items():
    if vardict["noy"]:
        base_cb = iris.load_cube(str(path_to_ukca / EXPS[base_exp_name] / f'{EXPS[base_exp_name]}_{ivar}.nc'), f'{ivar}')*VARS[ivar]['mmr_to_ppb']
        sens_cb = iris.load_cube(str(path_to_ukca / EXPS[sens_exp_name] / f'{EXPS[sens_exp_name]}_{ivar}.nc'), f'{ivar}')*VARS[ivar]['mmr_to_ppb']
        base_cb.rename(ivar)
        sens_cb.rename(ivar)
        base_cb_trimmed = base_cb[24::,...] # remove the first 2 years as a spin up
        sens_cb_trimmed = sens_cb[24::,...]
        iris.coord_categorisation.add_season(base_cb_trimmed, 'time', name='season')
        iris.coord_categorisation.add_season_year(sens_cb_trimmed, 'time', name='year')
        base_cl.append(base_cb_trimmed)
        sens_cl.append(sens_cb_trimmed)
# Horizontal grid
lons = base_cb.coord('longitude').points
lats = base_cb.coord('latitude').points

In [11]:
# Calculate NOy and RONO2
base_noy = base_cl[0].copy()
for cube in base_cl[1:]:
    base_noy += cube
base_noy.rename('noy')

rono2_cubes = base_cl.extract(['meono2', 'etono2', 'nprono2', 'iprono2'])
base_rono2 = rono2_cubes[0].copy()
for cube in rono2_cubes[1:]:
    base_rono2 += cube
base_rono2.rename('rono2')

In [ ]:
# Extract time series of boundary layer (0-2 km) seasonal means
rno3_bl_djf = (rno3.extract(iris.Constraint(season='djf')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
rno3_bl_mam = (rno3.extract(iris.Constraint(season='mam')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
rno3_bl_jja = (rno3.extract(iris.Constraint(season='jja')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
rno3_bl_son = (rno3.extract(iris.Constraint(season='son')).aggregated_by(['year', 'season'], iris.analysis.MEAN)[:,0:10,...]).collapsed('level_height', iris.analysis.MEAN)
# Calculate boundary layer seasonal means
rno3_bl_djf_mean = rno3_bl_djf.collapsed('year', iris.analysis.MEAN)
rno3_bl_mam_mean = rno3_bl_mam.collapsed('year', iris.analysis.MEAN)
rno3_bl_jja_mean = rno3_bl_jja.collapsed('year', iris.analysis.MEAN)
rno3_bl_son_mean = rno3_bl_son.collapsed('year', iris.analysis.MEAN)
# Add cyclic point for plotting on a global map
cyc_rno3_bl_djf_mean, cyclic_lons = cartopy.util.add_cyclic_point(rno3_bl_djf_mean.data, coord=rno3_bl_djf_mean.coord('longitude').points)
cyc_rno3_bl_mam_mean = cartopy.util.add_cyclic_point(rno3_bl_mam_mean.data)
cyc_rno3_bl_jja_mean = cartopy.util.add_cyclic_point(rno3_bl_jja_mean.data)
cyc_rno3_bl_son_mean = cartopy.util.add_cyclic_point(rno3_bl_son_mean.data)
# Find max boundary layer seasonal mean
print(np.max(rno3_bl_djf_mean.data)*1e3)
print(np.max(rno3_bl_mam_mean.data)*1e3)
print(np.max(rno3_bl_jja_mean.data)*1e3)
print(np.max(rno3_bl_son_mean.data)*1e3)
# rno3_mean_cf_kwargs = dict(transform=ccrs.PlateCarree(), levels=np.arange(0,110,10))

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12,12), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0)), facecolor='w')
p00 = ax[0,0].contourf(cyclic_lons, lats, cyc_sens_bl_djf_mean.data, transform=ccrs.PlateCarree())
ax[1,0].contourf(cyclic_lons, lats, cyc_sens_bl_mam_mean.data, transform=ccrs.PlateCarree())
ax[2,0].contourf(cyclic_lons, lats, cyc_sens_bl_jja_mean.data, transform=ccrs.PlateCarree())
ax[3,0].contourf(cyclic_lons, lats, cyc_sens_bl_son_mean.data, transform=ccrs.PlateCarree())

p01 = ax[0,1].contourf(cyclic_lons, lats, cyc_rno3_bl_djf_mean.data*100/cyc_sens_bl_djf_mean.data, transform=ccrs.PlateCarree(), levels=np.arange(0,110,10))
ax[1,1].contourf(cyclic_lons, lats, cyc_rno3_bl_mam_mean.data*100/cyc_sens_bl_mam_mean.data, transform=ccrs.PlateCarree(), levels=np.arange(0,110,10))
ax[2,1].contourf(cyclic_lons, lats, cyc_rno3_bl_jja_mean.data*100/cyc_sens_bl_jja_mean.data, transform=ccrs.PlateCarree(), levels=np.arange(0,110,10))
ax[3,1].contourf(cyclic_lons, lats, cyc_rno3_bl_son_mean.data*100/cyc_sens_bl_son_mean.data, transform=ccrs.PlateCarree(), levels=np.arange(0,110,10))

p01x = ax[0,1].contour(lons, lats, rno3_bl_djf_mean.data*100/sens_bl_djf_mean.data, transform=ccrs.PlateCarree(), levels=[10,50,80], colors='w')
p11x = ax[1,1].contour(lons, lats, rno3_bl_mam_mean.data*100/sens_bl_mam_mean.data, transform=ccrs.PlateCarree(), levels=[10,50,80], colors='w')
p21x = ax[2,1].contour(lons, lats, rno3_bl_jja_mean.data*100/sens_bl_jja_mean.data, transform=ccrs.PlateCarree(), levels=[10,50,80], colors='w')
p31x = ax[3,1].contour(lons, lats, rno3_bl_son_mean.data*100/sens_bl_son_mean.data, transform=ccrs.PlateCarree(), levels=[10,50,80], colors='w')
ax[0,1].clabel(p01x, inline=1, fmt='%1.0f')
ax[1,1].clabel(p11x, inline=1, fmt='%1.0f')
ax[2,1].clabel(p21x, inline=1, fmt='%1.0f')
ax[3,1].clabel(p31x, inline=1, fmt='%1.0f')

fig.subplots_adjust(hspace=0.1, wspace=-0.1)
cax00 = fig.add_axes([0.215, 0.1, 0.23, 0.01])
fig.colorbar(p00, cax=cax00, orientation='horizontal', label='$NO_y$, ppbv')
cax01 = fig.add_axes([0.58, 0.1, 0.23, 0.01])
fig.colorbar(p01, cax=cax01, orientation='horizontal', label='$\sum$$C_1$-$C_3$ $RONO_2$ % of $NO_y$')
fig.suptitle(f'Boundary layer (0-2 km)\n{sens_exp}', y=0.92, weight='bold')
for iax in ax.flatten(): iax.coastlines(color='k')